In [52]:
%pip install folium
import folium 
from folium.plugins import HeatMap
import json
import math
import pandas as pd 
import numpy as np
import requests

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:


url = "https://data.brisbane.qld.gov.au/api/explore/v2.1/catalog/datasets/traffic-data-at-intersection/exports/json?lang=en&timezone=Australia%2FBrisbane"

# Define the API key and the headers
api_key = "APIKEY"
headers = {
    "Authorization": f"Apikey {api_key}"
}

try:
    response = requests.get(url, headers=headers, verify=True)
    response.raise_for_status()  # Raises an HTTPError for bad responses (4xx or 5xx)

    data = response.json()
    
    # Convert JSON response to DataFrame
    PDTrafficVolume = pd.DataFrame(data)
  #  print(df.head())  # Print the first few rows of the DataFrame

except requests.exceptions.SSLError as ssl_err:
    print(f"SSL certificate verification failed: {ssl_err}")
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"An error occurred: {err}")

In [ ]:
#RawTrafficVolume = open('traffic-volume.json')
RawReference = open('intersections-reference.json')

JSONReference = json.load(RawReference)
#JSONTrafficVolume = json.load(RawTrafficVolume)

In [ ]:
#PDTrafficVolume = pd.DataFrame(JSONTrafficVolume)
PDReference = pd.DataFrame(JSONReference)

#remove records with no latlon
non_missing = PDReference[['latitude', 'longitude']].notna()
df_clean = PDReference[non_missing['latitude'] & non_missing['longitude']]


In [ ]:
merged_df = pd.merge(df_clean,PDTrafficVolume, how='left', left_on=['tsc'], right_on=['tsc'])

filtered_lat_df = merged_df[['latitude','longitude','tsc','lane','ds1']]

filtered_lat_df['ds1'] = filtered_lat_df['ds1'].fillna(0)


# Convert 'ds1' to numeric type
filtered_lat_df['ds1'] = pd.to_numeric(filtered_lat_df['ds1'])

group_means = filtered_lat_df.groupby('tsc').agg({'latitude': 'first', 'longitude': 'first', 'ds1': 'mean'}).reset_index()
#group_means = filtered_lat_df.groupby('tsc')['ds1'].mean()




#pd.set_option("display.max_rows", None)

group_means['ds1'] = group_means['ds1'] * 0.01

# Ensure all values in group_means['ds1'] are changed to 1 if they are greater than 1
group_means['ds1'] = np.where(group_means['ds1'] > 1, 1, group_means['ds1'])


In [58]:
#set up map
map_osm = folium.Map(location=[-27.467951, 153.020577],start_zoom = 16)

data = group_means[['latitude','longitude','ds1']]

HeatMap(data).add_to(map_osm)
map_osm